$$
Белорусский\;государственный\; университет
$$
$$
Механико-математический\;факультет
$$
$$
Кафедра\;дифференциальных\;уравнений\; и\; системного\; анализа
$$
$ $

$$
 \Large\bf Математические\; основы\; защиты\; информации
$$

# Тема 7. Применение криптосистемы RSA

$ $

доцент Чергинец Дмитрий Николаевич

# 1. Криптосистема RSA
**Генерация ключей в RSA**  
Input:  $\;L\in\mathbb{N}\;$ -- длина простых чисел.  
  
Output: $\;k_e\;$ -- открытый ключ,
		
$\qquad\;\;$   $\;k_d\;$ -- секретный  ключ.
		
		
1. Выбираем два случайных простых числа $\;p, q,\;$ состоящие из $\;L\;$ бит.
			
2. Вычисляем $\;n:=p q,\;$ $\;\varphi(n):=(p-1)(q-1).\;$
			
3. Случайным образом выбираем натуральное число $\;e,\;$ удовлетворяющее условиям
$$
1<e<\varphi(n),\qquad \gcd(e,\varphi(n))=1.
$$
4. При помощи расширенного алгоритма Евклида вычисляем 
$$
d:=e^{-1}\mod\varphi(n).
$$
5. Выдаем результат: $\;k_e:=(e,n),\;$ $\;k_d:=(d,n).$
			

## Шифрование и дешифрование в RSA
**Шифрование.**  
Функция шифрования 	
$$
\;E_{k_e}:\mathbb{Z}_n\rightarrow\mathbb{Z}_n\;
$$
определяется правилом
$$
 E_{k_e}(m):=m^e\mod n.
$$

**Дешифрование.**  
Функция дешифрования 	
$$
  D_{k_d}:\mathbb{Z}_n\rightarrow\mathbb{Z}_n
$$
задается формулой
$$
	D_{k_d}(c):=c^d\mod n.
$$

Отметим, что все три приведенных выше алгоритма являются полиномиальными.

# 2. PKCS (Public Key Cryptography Standards)

- RSA Security  -- компания, основанная авторами криптосистемы RSA: Рональдом Ривестом, Ади Шамиром и Леонардом Адлеманом;
	
- RSA Security разработала ряд стандартов под названием PKCS (Public Key Cryptography Standards);
	
- Рассмотрим стандарт [PKCS \#1, версия 2.2,](http://mpqs.free.fr/h11300-pkcs-1v2-2-rsa-cryptography-standard-wp_EMC_Corporation_Public-Key_Cryptography_Standards_(PKCS).pdf)   разработанный в 2012 году.

## Перевод текста в число
**OS2IP (Octet String 2 Integer Primitive)**  
Input: $\;X\;$ - строка, состоящая из $\;XLen\;$ байт:
$$
X=\{X_{XLen-1}, X_{XLen-2},\dots,X_0\},\qquad 0\leq X_i<256.
$$ 
Output:  $\;x\in\mathbb{N}\cup\{0\}.$ 
 1. Выдаем результат:
$$
  x:=X_{XLen-1}256^{XLen-1}+X_{XLen-2}256^{XLen-2}+\dots+X_{0}.
$$  
		

In [1]:
def OS2IP(X: bytes) -> int:
    x = 0
    for i in range(len(X)):
        x += X[i] * 256**(len(X)-1-i)
    return x

message = b'Hello Bob!'  # Python не знает, как представлять байтовую строку
x = OS2IP(message)
print(x)

OS2IP(message) == int.from_bytes(message, 'big')

# возвращает целое число, представленное заданным массивом байтов
# eсли byteorder имеет значение 'big', то самый значимый байт находится в начале массива байтов

341881320659697045299745


True

При индексации по байтовой строке получаем код соответствующего символа в таблице ASCII.

## Перевод числа в байты
**I2OSP (Integer to Octet String Primitive)**  
Input: $x\in\mathbb{N}\cup\{0\};$  
$\qquad XLen$ -- длина байтовой строки.  

Output: 
$$X=\{X_{XLen-1}, X_{XLen-2},\dots,X_0\},\qquad 0\leq X_i<256.$$ 

1.  Если $\;x\geq 256^{XLen},\;$ то выдаем результат: "Число слишком большое".  
	
2. Переводим $\;x\;$ в систему счисления по основанию 256:
$$
  x:=X_{XLen-1}256^{XLen-1}+X_{XLen-2}256^{XLen-2}+\dots+X_{0}.
$$  
		
3. Выдаем результат:
$$
  X=\{X_{XLen-1}, X_{XLen-2},\dots,X_0\}.
$$
		
		

In [2]:
def I2OSP(x: int, XLen: int) -> bytes:
    if x >= 256**XLen:
        raise ValueError("Number %d must be less than %d" % (x,256**XLen))
    b = []
    for _ in range(XLen):
        b.append(x % 256)
        x >>= 8
    b.reverse()
    return bytes(b)

plaintext = I2OSP(x, len(message)) 
print(plaintext)

plaintext == x.to_bytes(len(message), "big")

b'Hello Bob!'


True

## Необходимость дополнять сообщение
Следующие сообщения шифруются плохо.  
**Сообщение небольшого размера.**  
$$
  0^e\equiv 0 \mod n;
$$ 
$$
  1^e\equiv 1 \mod n;
$$ 
$$
  2^3\equiv 8 \mod n.
$$

**Дополнение должно быть вероятностным.**

Если
$$
  m\in\{m_1, m_2, \dots, m_s\},\qquad s - \text{невелико},
$$
тогда зная шифротекст
$$
   c:= m^e\mod n
$$
открытый текст $\;m\;$ можно найти перебором.   
 	
 	

## Message padding
В стандарте PKCS имеется два алгоритма дополнения (или можно ещё сказать кодирования) открытого текста: 
- Алгоритм   PKCS1-v1.5;
- Алгоритм OAEP.

Слово "padding" означает набивка, набивочный материал, ватин. Слово "дополнение" не совсем точно отражает суть процесса.
 

 ## Алгоритм дополнения сообщения PKCS 
**Алгоритм   PKCS1-v1.5**   
Input: $\;m\;$ - сообщение из $\;mLen\;$ байт;

$\qquad k$ -- количество байт в модуле $\;n\;$  $(mLen\leq k-11).$

Output:  $\;EM\;$ (encoded message) - закодированное сообщение, состоящее из $\;k\;$  байт. 

1$.$  Если $\;mLen>k-11,\;$ выдаем ошибку: "сообщение слишком велико", конец алгоритма.

2$.$ Генерируем случайный список $\;PS\;$ (Pseudorandom String) из $\;k-mLen-3\;$ ненулевых байт.


3$.$  Выдаем результат: 
$$
  EM:=0||2||PS||0||m.
$$  


 

## Генерация случайных байт
Для генерации случайных чисел раньше мы пользовались модулем `ramdom`. Но говорят, что для криптографии он не годится, поэтому случайные байты в алгоритме PKCS мы будем генерировать при помощи метода `urandom` модуля `os`

In [3]:
import os, time
retries = 100_000

start_time = time.time()
rb = os.urandom(retries)  # возвращает объект bytes, содержащий случайные байты, подходящие для криптографического использования
print("Всё за раз:           --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(retries):
    os.urandom(1)

print("По 1 байту много раз: --- %s seconds ---" % (time.time() - start_time))

Всё за раз:           --- 0.0 seconds ---
По 1 байту много раз: --- 0.03017115592956543 seconds ---


In [4]:
def EncodePKCS(m: bytes, k: int) -> bytes:
    mLen = len(m)
    if mLen > k - 11:
        raise ValueError("Message length must be less than k-10")
    PS = b""
    while len(PS) < k - mLen - 3:
        rb = os.urandom(1)
        if not rb == b'\x00':
            PS += rb
    return b'\x00\x02' + PS + b'\x00' + m        

EM = EncodePKCS(message, 30) 
EM

b'\x00\x02\x0cXr\xc4\xf8\t\xa3\xf6\xdd\xf7\xae\x06\x86js!\x9c\x00Hello Bob!'

'\x00' - представление нулевого байта (байта без информации).

Срезы поддерживаются байтовыми строками (возвращается байтовая строка).

## Зашифруем сообщение с дополнением PKCS
Попросим у Боба открытый ключ

In [5]:
import rsa
(bob_pub, bob_priv) = rsa.newkeys(512)
e = bob_pub.e
print('e = ', e)
n = bob_pub.n
print('n = ', n)

e =  65537
n =  7436699190607040911919166270529218998116027006104293859532057010057251532428507157277501825566316612068197244049739674100359625851745183518926017881737219


### Подсчитаем из скольких байт состоит модуль

In [6]:
import math
def Len(n: int) -> int:
    if type(n) == int:
        return math.ceil((len(bin(n)) - 2) / 8)
    s = 0
    for i in n:
        s += Len(i)
    return s    

k = Len(n)
print('Модуль n состоит из  %d  байт' % k)

Модуль n состоит из  64  байт


## Вычисляем шифротекст и отправляем его Бобу

In [7]:
EM = EncodePKCS(message, k)  # набиваем дополнительные байты
m = OS2IP(EM)  # преобразуем полученную байтовую строку в число реализованным ранее алгоритмом
c = pow(m, e, n)  # шифруем число
CipherText = I2OSP(c, k)  # преобразуем полученное число в байтовую строку
CipherText

b'\x8a`M\xcf\x07\xab\x89B\x7f\xb4\xe2N\xf6\xc7\x04D\x86\xae\xaehh\x85\xb3\xe7kV\x8d\x06\x12\xd6\xaa\x85>}h\xb8>\xea^u{\x8f\xa5\xbb\xe9\x91\t\xab\x84P\xff\x18N\x7f\xb1mt"\x97\xd8R\x96\xc29'

## Боб расшифровывает сообщение
Боб получает от нас шифротекст и расшифровывает сообщение, используя пакет RSA. 

In [8]:
PlainText = rsa.decrypt(CipherText, bob_priv)
PlainText 

b'Hello Bob!'

Несмотря на то, что шифротекст CipherText получен без использования пакета RSA, несовместимости програмного обеспечения не произошло и Боб получил открытый текст.

## <font color='red'>Задание 1.</font>
Реализовать алгоритм DecodePKCS, который по дополненной согласно алгоритму PKCS строке байт EM находит открытый текст. Для EM1, EM2, EM3, где это возможно, найти текст m1, m2, m3, закодированный кодировкой 'utf-8'.

In [9]:
EM1 = b'\x00\x02\xad\xed\x08\xb4\x95\xb6c\xc3\xb9\x08\x81\xb8\xd5\xbb\r\x04\x96\x00\xd0\xa0\xd0\xb0\xd0\xb1\xd1\x96 \xd0\xbd\xd0\xb5\xd1\x87\xd0\xb0\xd0\xba\xd0\xb0\xd0\xbd\xd0\xb0\xd0\xb5, \xd1\x80\xd0\xb0\xd0\xb1\xd1\x96, \xd1\x8f\xd0\xba \xd0\xbd\xd0\xb5 \xd0\xb1\xd1\x8b\xd0\xb2\xd0\xb0\xd0\xb5, \xd1\x80\xd0\xb0\xd0\xb1\xd1\x96, \xd1\x8f\xd0\xba \xd0\xbd\xd0\xb5 \xd1\x80\xd0\xbe\xd0\xb1\xd1\x96\xd1\x86\xd1\x8c \xd0\xbd\xd1\x96\xd1\x85\xd1\x82\xd0\xbe, \xe2\x80\x94 \xd1\x96 \xd1\x82\xd0\xb0\xd0\xb4\xd1\x8b \xd0\xbf\xd0\xb5\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xb6\xd0\xb0\xd1\x88.'
EM2 = b'\x00\x02r8\x00\xd0\xa7\xd0\xb0\xd0\xbb\xd0\xb0\xd0\xb2\xd0\xb5\xd0\xba \xd0\xbd\xd0\xbe\xd1\x81\xd1\x96\xd1\x86\xd1\x8c \xd1\x81\xd0\xb2\xd0\xb0\xd1\x91 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xb0 \xd0\xb7 \xd1\x81\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x8e.'
EM3 = b'\x02\x02\xf5\xb2\xf2\xb2LG\x9b\r\xe4\x00\xd0\xa7\xd0\xb0\xd0\xbb\xd0\xb0\xd0\xb2\xd0\xb5\xd0\xba \xd0\xbd\xd0\xbe\xd1\x81\xd1\x96\xd1\x86\xd1\x8c \xd1\x81\xd0\xb2\xd0\xb0\xd1\x91 \xd0\xbd\xd0\xb5\xd0\xb1\xd0\xb0 \xd0\xb7 \xd1\x81\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x8e.'

Поскольку сообщение закодировано кодировкой 'utf-8', его необходимо только найти и расшифровать, используя метод decode('utf-8').

In [10]:
def DecodePKCS(EM: bytes) -> str:
    counter = 0
    
    for i in range(len(EM)):
        if EM[i] == 0:
            counter += 1
        if counter == 2:
            cipher_text = EM[i + 1:]  # зашифрованное сообщение в виде байтовой строки
            return cipher_text.decode('utf-8')
        
    return 'The message could not be decrypted'

In [11]:
DecodePKCS(EM1)

'Рабі нечаканае, рабі, як не бывае, рабі, як не робіць ніхто, — і тады пераможаш.'

In [12]:
DecodePKCS(EM2)

'Чалавек носіць сваё неба з сабою.'

In [13]:
DecodePKCS(EM3)

'The message could not be decrypted'

## <font color='red'>Задание 2.</font>
Сообщение m зашифровано при помощи модуля RSA. Найти сообщение m, не пользуясь функциями модуля RSA.

In [14]:
d = 26770772907582938053130151905548181178036083280747414634624384031357480520230808847626726751112707871133365466932082860352116479532095585837114011518890329506258186862891674713345047774452260967801004643838706727945877863169543110385438010586368708217542522729977757884858964820726738896465984173006606967577
n = 101632169613871459838266278481950480789257417017340167578600373994269547636816689998662618958910591192172123767961994694948540677697674124370442075073767460922116257699815386922669167387324184470375309591093269335567795836909746438396358314576200234449897268393760086784065030629672726231457824423660894104727
CipherText = b"\x18\xf0\xb1\x94'\xfb\xd2\xcc\x0e\x00\x08\xe3\xf3\x13T\xfdE\x0fE\x87N\xc6\xe0\xb01X\x9d!\xf1\x06\x16{\xd8\xce+\xc8\xbf\x9e$\xbb\xfd\xdb\xb1-\xdf4(v\x1d\\\x1d\xe4\x8cVC=\xbex\xf6\x08\x95\rvX\x92\xd9#S\xa7\xd9-r\x84q\x8f\xe1\x96[x\x03\xc5\x0f\xed\x9f\x0b\xd8\xfb\x07\x97\xef\xa3\xfcQ\xf8\x98\x8c\x89\x15\x84\x90?\xd4\xf3\x96\x05\xd1b\xear\x84v\x81m\x0b\xcdD\x14L\xe1e]\xc7H\x99\x13>E\xc3"

Поскольку нам известен приватный ключ, можно преобразовать число $c$ в число $m$, которое будет отображать исходное сообщение. Подействуя функцией I2OSP на число $m$, получим байтовую строку, которая будет по сути являться исходным сообщением. Однако, видим, что исходное сообщение зашифровано с дополнением PKCS, поэтому нужно вызвать функцию DecodePKCS.

In [15]:
c = OS2IP(CipherText)
m = pow(c, d, n)
EM = I2OSP(m, len(CipherText))
DecodePKCS(EM)

'Вечна тваім застанецца толькі тое, што ты аддаў'

# 3. Дешифрование при помощи CRT
Функция дешифрования задается формулой
$$
	D_{k_d}(c):=c^d\mod n.
$$
Заметим, что открытый текст 
$$
   m :=   c^d\mod n
$$
удовлетворяет системе уравнений
$$
  \left\{\begin{array}{ll}
 m \equiv c^d \mod p;\\
 m \equiv c^d \mod q.\\
 \end{array}\right.
$$

Поэтому вместо вычисления $\quad c^d\mod n\quad$ можно попробовать вычислять 
$$
  m_p:=c^d\mod p,\qquad m_q:=c^d\mod q,
$$
после чего находить $\;m\;$ из системы
$$
  \left\{\begin{array}{ll}
 m \equiv m_p \mod p;\\
 m \equiv m_q \mod q.\\
 \end{array}\right.\qquad (*)
$$

## Секретный ключ при дешифровании CRT
Так как 
$$
  c^{p-1}\equiv 1\mod p,
$$
то
$$
  m_p:=c^d\equiv c^{dP}\mod p,\qquad m_q:=c^d\equiv c^{dQ}\mod q,
$$
где 
$$
  dP:=d\mod p-1,\qquad dQ:=d \mod q-1.
$$

Согласно алгоритму Гарнера решение системы $\;(*)\;$ находится по формулам
$$
  y:=(m_p-m_q)*qInv \mod p,\qquad m:=m_q+q*y.
$$
Здесь 
$$
qInv:=q^{-1}\mod p.
$$

Таким образом, секретный ключ при дешифровании Китайской теоремой об остатках имеет вид:

$$
  PrivateKey:=(p,q,dP,dQ,qInv).
$$

## <font color='red'>Задание 3.</font>
Реализовать функцию генерации ключей криптосистемы RSA. Секретный ключ должен иметь вид
$$
  PrivateKey:=(p,q,dP,dQ,qInv).
$$
Можно ли ускорить не только дешифрование, но и шифрование при помощи Китайской теоремы об остатках?

In [16]:
from random import randint
import math

def st(n):
    s = 0
    t = n - 1
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t

def WitnessQ(a, n, s = 0, t = 0):  
    if n % 2 == 0:
        return False
    if s == 0:
        s, t = st(n)    
    if math.gcd(a,n) > 1:
        return False
    b = pow(a,t,n)
    if b == 1 or b == n - 1:  # модуль - число положительное
        return True
    for _ in range(1,s):
        b = b**2 % n
        if b == n - 1:
            return True
    return False

def MillerRabinTest(n, k = 10):
    if n % 2 == 0:
        return False
    s,t = st(n)
    for _ in range(k):
        a = randint(2,n - 1)
        if not WitnessQ(a,n,s,t):
            return False
    return True

def generates_prime_number(b):
    while True:
        n = randint(2**(b - 1),2**b - 1)
        if MillerRabinTest(n):
            return n
        else:
            continue
            
def ExtendedGCD(a, b):  # расширенный алгоритм Евклида
    m11, m12 = 1, 0
    m21, m22 = 0, 1
    while b:
        q = a // b
        r = a % b
        a, b = b, r
        m11, m12 = m12, m11 - m12*q
        m21, m22 = m22, m21 - m22*q 
    return (m11,m21)

def RSA_key_generation(L: int) -> dict:
    
    """
    L is the length of the primes
    
    k_e is the open key
    k_d is the close key
    """
    
    p, q = generates_prime_number(L), generates_prime_number(L)
    n = p * q
    phi = (p - 1) * (q - 1)
    
    while True:
        e = randint(1 + 1, phi - 1)  # чтобы обеспечить выполнение неравенства
        
        if math.gcd(e, phi) == 1:
            break
        else:
            continue
            
    d, _ = ExtendedGCD(e, phi)  # обосновать через коэффициенты Безу
    
    return {'k_e': (n, e), 'k_d': (n, d)}

def private_key_search_p_q(key_e: tuple, d: int) -> tuple:
    
    """ search p and q using a private key """
    
    n, e = key_e
    s, t = st(e * d)  # не вычитаем 1, так как это уже сделано в теле функции st
    
    while True:
        a = randint(1 + 1, n - 2)

        if math.gcd(a, n) > 1:
            p = math.gcd(a, n)
            q = n // p

            return (p, q)
        else:
            u = pow(a, t, n)
            v = pow(u, 2, n)

            if u == 1:
                continue
            else:
                while v != 1:
                    u = v
                    v = pow(u, 2, n)
                
                if (u + 1) % n == 0:
                    continue
                else:
                    p = math.gcd(u + 1, n)
                    q = math.gcd(u - 1, n)
                    
                    return (p, q)

def RSA_key_generation_CRT(L: int) -> dict:
    
    """
    L is the length of the primes
    
    k_e is the open key
    k_d_CRT is the close key for CRT
    k_d is the close key
    """
    
    keys = RSA_key_generation(L)
    
    d = keys['k_d'][1]
    
    (p, q) = private_key_search_p_q(keys['k_e'], d)
    
    dP = d % (p - 1)
    dQ = d % (q - 1)
    qInv = pow(q, -1, p)
    
    return {'k_e': keys['k_e'], 'k_d_CRT': (p, q, dP, dQ, qInv), 'k_d': keys['k_d']}

In [17]:
RSA_key_generation_CRT(100)

{'k_e': (794963321865710157453349647558377121886028409631717358244721,
  638909607144664735744980479058952738737590329054503727678261),
 'k_d_CRT': (1168920117910562290273431885127,
  680083531530539772039578668423,
  1024984010079902481183595137607,
  409254699051121587759972352273,
  1079483066891238206058506520189),
 'k_d': (794963321865710157453349647558377121886028409631717358244721,
  -189105570545063283430199225168214936033547204219285594968095)}

In [18]:
def encryption_E(m: int, k_e: tuple) -> int:
    
    """ encryption with method RCA """
    
    return pow(m, k_e[1], k_e[0])

def decryption_D(c: int, k_d: tuple) -> int:
    
    """ decryption with method RCA """
    
    return pow(c, k_d[1], k_d[0])

In [19]:
def DecryptCRT(c, kd):
    (p,q,dP,dQ,qInv) = kd
    mp = pow(c,dP,p)
    mq = pow(c,dQ,q)
    y  = (mp-mq)*qInv % p 
    return mq + q*y

message = 121
L = 100

keys = RSA_key_generation_CRT(L)

ke, kdCRT = keys['k_e'], keys['k_d_CRT']
kd = keys['k_d']

CipherText = encryption_E(message, ke)
PlainText = DecryptCRT(CipherText, kdCRT)

PlainText == message

True

In [20]:
import time
retries = 80

start_time = time.time()
for _ in range(retries):
    DecryptCRT(CipherText, kdCRT)
print("DecryptCRT:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(retries):
    decryption_D(CipherText, kd)
print("Decrypt:--- %s seconds ---" % (time.time() - start_time))

DecryptCRT:--- 0.010765552520751953 seconds ---
Decrypt:--- 0.013241052627563477 seconds ---


Проверим, можно ли ускорить не только дешифрование, но и шифрование при помощи китайской теоремы об остатках, непосредственно.

In [21]:
def encryption_СRT(m: int, k_e_CRT: tuple) -> int:
    
    """ encryption with method RCA with help CRT """
    
    (p, q, dP, dQ, qInv) = k_e_CRT
    
    c_p = pow(m, dP, p)
    c_q = pow(m, dQ, q)
    y = (c_p - c_q) * qInv % p 
    
    return c_q + q * y

In [22]:
start_time = time.time()
for _ in range(retries):
    encryption_СRT(message, kdCRT)
print("encryption_СRT:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(retries):
    encryption_E(message, ke)
print("encryption_E:--- %s seconds ---" % (time.time() - start_time))

encryption_СRT:--- 0.009376049041748047 seconds ---
encryption_E:--- 0.009509086608886719 seconds ---


Видим, что с помощью Китайской теоремы об остатках можно ускорить и шифрование.

## Длина секретного ключа
Сравним количество байт обычного секретного ключа $\,(e,n)\,$ с длиной секретного ключа для дешифрования при помощи Китайской теоремы об остатках $\;(p,q,dP,dQ,qInv)$

In [23]:
keys = RSA_key_generation_CRT(1024)
kd, kdCRT = keys['k_d'], keys['k_d_CRT']

print('Длина обычного секретного ключа = ', Len(kd))
print('Длина секретного ключа для CRT  = ', Len(kdCRT))

keys = RSA_key_generation_CRT(2048)
kd, kdCRT = keys['k_d'], keys['k_d_CRT']

print('Длина обычного секретного ключа = ', Len(kd))
print('Длина секретного ключа для CRT  = ', Len(kdCRT))

Длина обычного секретного ключа =  512
Длина секретного ключа для CRT  =  640
Длина обычного секретного ключа =  1024
Длина секретного ключа для CRT  =  1279


# 4. Дополнение сообщения алгоритмом OAEP

- Optimal Asymmetric Encryption Padding

- Оптимальное дополнение асимметричного шифрования;

- Авторы Mihir Bellare, Phillip Rogawayy, 1994 год; 

- Рассмотрим модификацию OAEP, описанную в стандарте PKCS$\#1.$


## Функция генерации маски
**Алгоритм	MGF1 (Mask generation function)**
		
Input: 	$\;seed\;$ -- строка байт произвольной длины;
			
$\qquad maskLen$ -- количество байт в значении функции MGF1;
			
$\qquad Hash$ --  хеш-функция со значением hLen байт.

Output: $mask$  -- строка, состоящая из maskLen байт.

1$.$  Создаем пустую байтовую строку $\;T:=\{\}.$
			
2$.$  Для $\;i:=0,\dots, \lceil maskLen/hLen)\rceil-1\;$ выполняем:
			
$\qquad$ 2.1.  Представляем $\;i\;$  в виде четырех байт: 
$$
  C:=\text{I2OSP}(i,4).
$$
$\qquad$ 2.2.  Вычисляем $\;  T:=T||Hash(seed||C).$
			
3$.$  Выдаем результат: первые maskLen байт переменной $\;T$
$$
\;mask:=T[0:maskLen].\;
$$

In [24]:
# дайджест, или хэш-сумма - это значение фиксированной длины, вычисляемое по большому набору данных

def MGF1(seed: bytes, maskLen: int, MaskHash) -> bytes:
    T = b''
    hLen = MaskHash.digest_size
    for i in range(int(maskLen / hLen) + 1):  # + 1, чтобы использовать int, а не math.ceil
        C = I2OSP(i, 4)
        h = MaskHash.copy()  # возвращает копию хешируемого объекта
        h.update(seed + C)  # обновляет состояние хэш-объекта с помощью предоставленной строки
        T += h.digest()  # возвращает значение дайджеста в виде байтового объекта
    return T[:maskLen]    

import hashlib    
Hash = hashlib.sha256()
seed = b'seed'
maskLen = 15
mask = MGF1(seed, maskLen, Hash)

list(mask)

[51, 111, 40, 160, 34, 25, 57, 57, 88, 90, 27, 78, 220, 152, 159]

##  Кодирование сообщения методом OAEP

![Для отображения картинки нужен интернет](https://drive.google.com/uc?export=view&id=1c0KMWiCd4p1i_uBMOW0hte_a4qIE9OEg)



##  Кодирование сообщения методом OAEP
**Encoding  RSAES-OAEP**  
Options:  
$\qquad$ $Hash$ --  хеш-функция с длиной дайджеста $\;hLen\;$ байт;  
$\qquad$ $MGF$ -- функция генерации маски.

Input:   
$\qquad k\;$ -- количество байт в модуле $\;n\;$ шифра RSA.

$\qquad m$ -- сообщение, состоящее из mLen байт, $\; mLen\leq k-2hLen-2.$
	
$\qquad L$ -- label, метка, строка байт.  В некотором смысле может выступать дополнительным секретным ключом.   
$\quad\qquad$ Если $\;L\;$ не получена, то $L=\{\}.$
	
Output:   
$\qquad$ $EM$ -- дополненное сообщение длины $\;k\;$  байт.



1$.$  Вычисляем хеш от метки $\;\;LHash:=Hash(L).$

2$.$  Генерируем байтовую строку $\;PS,\;$ состоящую из  $\;k-mLen-2hLen-2\;$ нулей. 
Данный список может оказаться пустым.

3$.$ Образуем байтовую строку $\;DB\;$ (Data Block) длины $\;k-hLen-1\;$
$$
  DB:=LHash||PS||1||m.
$$  

4$.$  Генерируем случайную байтовую строку $\;seed\;$ длины $\;hLen\;$  байт.

5$.$ Вычисляем маску
$$
  dbMask:=MGF(seed,k-hLen-1).
$$
6$.$ Прячем под маской переменную $\;DB$
$$
   maskedDB:=DB\oplus dbMask,
$$  
где $\oplus$ -- XOR.
            
7$.$ Вычисляем маску
$\quad
  seedMask:=MGF (maskedDB,hLen).
$  
8$.$ Прячем seed под маской   
$$
  maskedSeed:=seed\oplus seedMask.
$$
9$.$ Выдаем результат $\;EM:=0||maskedSeed||maskedDB.$

In [25]:
def byte_xor(b1, b2):
    return bytes([a ^ b for a, b in zip(b1, b2)])

def EncodeOAEP(k, m, L = b'', Hash = hashlib.sha256(), MaskHash = hashlib.sha256()):
    Hash.update(L)
    LHash = Hash.digest()
    mLen = len(m)
    hLen = Hash.digest_size
    PS = (k - mLen - 2 * hLen - 2) * b'\x00'
    DB = LHash + PS + b'\x01' + m
    seed = os.urandom(hLen)
    dbMask = MGF1(seed, k - hLen - 1, MaskHash)
    maskedDB = byte_xor(DB, dbMask) 
    seedMask = MGF1(maskedDB, hLen, MaskHash)
    maskedSeed = byte_xor(seed, seedMask)
    return b'\x00' + maskedSeed + maskedDB

EM = EncodeOAEP(2048, b'Hello Bob!')

## Методы дополнения в пакете RSA
В пакете RSA реализован лишь 1 метод дополнения сообщения из стандарта PKCS

In [26]:
help(rsa.encrypt)

Help on function encrypt in module rsa.pkcs1:

encrypt(message: bytes, pub_key: rsa.key.PublicKey) -> bytes
    Encrypts the given message using PKCS#1 v1.5
    
    :param message: the message to encrypt. Must be a byte string no longer than
        ``k-11`` bytes, where ``k`` is the number of bytes needed to encode
        the ``n`` component of the public key.
    :param pub_key: the :py:class:`rsa.PublicKey` to encrypt with.
    :raise OverflowError: when the message is too large to fit in the padded
        block.
    
    >>> from rsa import key, common
    >>> (pub_key, priv_key) = key.newkeys(256)
    >>> message = b'hello'
    >>> crypto = encrypt(message, pub_key)
    
    The crypto text should be just as long as the public key 'n' component:
    
    >>> len(crypto) == common.byte_size(pub_key.n)
    True



# 5. Пакет cryptography
В пакете cryptography есть реализация стандарта PKCS c двумя способами дополнения:
- PKCS#1 v1.5;
- OAEP

In [27]:
import math
from cryptography.hazmat.primitives.asymmetric import rsa as rsaHazMat

private_key = rsaHazMat.generate_private_key(public_exponent=65537,key_size=1024)

public_key = private_key.public_key()
pn = public_key.public_numbers()
e = pn.e
n = pn.n
print('e = ', e)
print('n = ', n)
k = Len(n)
print('Модуль n состоит из  %d  байт' % k)

e =  65537
n =  120308445575934149764476393919601332921487584722439375790798406384939668897322716663870307490898720401601321271863809003443477948321926620438852582822318257978508898738889133058499628067502800242230627248829115433916893522631596211206312737162040503362943857934765668987989469497013008992035795171500658756773
Модуль n состоит из  128  байт


## Шифрование с применением дополнения PKCS
Зашифруем открытый текст `message` при помощи нашей функции дополнения `EncodePKCS` и открытого ключа Боба

In [28]:
message = b"Hello Bob!"
EM = EncodePKCS(message, k) 
m = OS2IP(EM)
c = pow(m, e, n) 
CipherText = I2OSP(c, k)

#### Боб расшифровывает сообщение при помощи метода `decrypt` класса `private_key` из пакета cryptography

In [29]:
from cryptography.hazmat.primitives.asymmetric import padding
plaintext = private_key.decrypt(
    CipherText,
    padding.PKCS1v15()
)
print(plaintext)
plaintext == message

b'Hello Bob!'


True

## Шифрование с применением дополнения OAEP
Зашифруем открытый текст `message` при помощи нашей функции дополнения `EncodeOAEP` и открытого ключа Боба

In [30]:
import hashlib
Hash = hashlib.sha256()
MaskHash = hashlib.sha1()
message = b"Hello  KM!"
L = b'seed'
EM = EncodeOAEP(k, message, L, Hash, MaskHash)
m = OS2IP(EM)
c = encryption_E(m, (n, e))
CipherText = I2OSP(c, k)
len(CipherText)

128

#### Боб расшифровывает сообщение при помощи пакета `cryptography`
Для того чтобы пользоваться дополнением OAEP Бобу необходимо знать параметры дополнения:
- Hash - функция хеширования, используемая в алгоритме OAEP;
- MaskHash - функция хеширования, используемая в алгоритме OAEP;
- Метку L.

Отметим, что при дешифровании используются функции хеширования из модуля `hashes`, а при шифровании функции хеширования принадлежали пакету `hashlib`

In [31]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

plaintext = private_key.decrypt(
    CipherText,
    padding.OAEP(
        mgf=padding.MGF1(algorithm = hashes.SHA1()),
        algorithm = hashes.SHA256(),
        label=L
    )
)
print(plaintext)
plaintext == message

b'Hello  KM!'


True

## Пакет `hashes`
Вычислять дайджест можно и при помощи пакета `hashes`.

In [32]:
print(dir(hashes.Hash))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'algorithm', 'copy', 'finalize', 'update']


In [33]:
h = hashes.Hash(hashes.SHA256())
m = b'Test'
h.update(m)
digest = h.finalize()
print(digest)
hashlib.sha256(m).digest() == digest

b'S.\xaa\xbd\x95t\x88\r\xbfv\xb9\xb8\xcc\x00\x83, \xa6\xec\x11=h"\x99U\rzn\x0f4^%'


True

#  6. Декодирование сообщения методом OAEP

![Для отображения картинки нужен интернет](https://drive.google.com/uc?export=view&id=1c0KMWiCd4p1i_uBMOW0hte_a4qIE9OEg)

##  Дедирование сообщения методом OAEP
**Decoding  RSAES-OAEP**  
Options:  
$\qquad$ $Hash$ --  хеш-функция с длиной дайджеста $\;hLen\;$ байт;  
$\qquad$ $MGF$ -- функция генерации маски.

Input:   
$\qquad k\;$ -- количество байт в модуле $\;n\;$ шифра RSA.

$\qquad EM$ -- дополненное сообщение, состоящее из $\;k\;$ байт.
	
$\qquad L$ -- label, метка, строка байт.   
$\quad\qquad$ Если $\;L\;$ не получена, то $L=\{\}.$
	
Output:   
$\qquad$ $m$ --  сообщение.

1$.$ Разделяем $\;EM\;$ на три переменные $\;Y,\;$  $\;maskedSeed,\;$ $\;maskedDB\;$ длин $\;1, hLen, k-hLen-1,\;$ соответственно
$$  (Y, maskedSeed, maskedDB):=EM. $$

2$.$ Снимая маску, находим росток
$$
seed:=maskedSeed\oplus MGF(maskedDB,hLen).
$$

3$.$  Снимая маску, находим
$$
DB=maskedDB\oplus MGF(seed,k-hLen-1).
$$

4$.$  Разделяем $\;DB\;$ на переменные $\;LHash, PS, 1, m,\;$ по правилу
$$
(LHash, PS, 1, m):=DB,
$$
$\quad$ где $\;LHash\;$ длины $\;hLen,\;$
			
$\quad 1$ -- первая единица в $\;DB,\;$ начиная с позиции $\;hLen,$ если такой единицы нет, то выдаем ошибку.
			
5$.$ Если не выполнено хотя бы одно из условий:
$$Y=0,\quad LHash=Hash(L),\quad  PS\;\; \text{ состоит из нулей,}$$
    то выдаем ошибку:  $\;EM\;$ не получено при помощи Encoding RSAES-OAEP.
			
			
6$.$   Выдаем результат $\;m.$

In [34]:
def DecodeOAEP(k, EM: bytes, L: bytes = b'', Hash = hashlib.sha256(), MaskHash = hashlib.sha256()) -> str:
    hLen = Hash.digest_size

    Y, maskedSeed, maskedDB = EM[0], EM[1:hLen + 1], EM[hLen + 1:]

    seed = byte_xor(maskedSeed, MGF1(maskedDB, hLen, MaskHash))
    DB = byte_xor(maskedDB, MGF1(seed, k - hLen - 1, MaskHash))

    LHash = DB[0:hLen]

    PS = list()
    counter = -1

    for i in range(hLen, len(DB)):
        if DB[i] == 1:
            counter = i + 1
            break
        else:
            PS.append(0)

    if counter == -1:  # проверка на отсутствие 1 
        return 'Error'

    Hash.update(L)
    HL = Hash.digest()
    if Y != 0 or LHash != HL:
        return 'Error'

    for element in PS:
        if element != 0:
            return 'Error'
        else:
            continue

    return DB[counter:]

# 7. Реализация RSA, используя ООП

## <font color='red'>Задание 4.</font>
Реализовать класс PublicKey с методами шифрования, использующими PKCS и OAEP. Объект класса PublicKey генерируется по открытой экспоненте `e` и модулю `n`.

In [35]:
class PublicKey:  
    def __init__(self, ke: tuple):
        self.ke = ke
    
    def encode_PKCS(self, message: bytes) -> bytes:
        
        """ encoding with method PKCS """
        
        k = Len(self.ke[0])
        
        EM = EncodePKCS(message, k)
        m = OS2IP(EM)
        c = encryption_E(m, self.ke)
        
        return I2OSP(c, k)
    
    def encode_OAEP(self, message: bytes, L: bytes = b'', Hash = hashlib.sha256(), MaskHash = hashlib.sha256()) -> bytes:
        
        """ encoding with method OAEP """
        
        k = Len(self.ke[0])
        
        EM = EncodeOAEP(k, message, L, Hash, MaskHash)
        m = OS2IP(EM)
        c = encryption_E(m, self.ke)
        
        return I2OSP(c, k)

## <font color='red'>Задание 5.</font>
Реализовать класс PrivateKey с методами дешифрования, использующими PKCS и OAEP, с методом генерации открытого ключа в виде класса PublicKey. Дешифрование должно проходить при помощи Китайской теоремы об остатках. 

In [36]:
class PrivateKey:
    def __init__(self, kdCRT: tuple, kd: tuple):
        self.kdCRT = kdCRT
        self.kd = kd
        
    def decode_PKCS(self, cipher_text: bytes) -> str:
        
        """ decoding with method PKCS """
        
        c = OS2IP(cipher_text)
        m = DecryptCRT(c, self.kdCRT)
        EM = I2OSP(m, len(cipher_text))
        
        return DecodePKCS(EM)
    
    def decode_OAEP(self, cipher_text: bytes, L: bytes = b'', Hash = hashlib.sha256(), MaskHash = hashlib.sha256()) -> str:
        
        """ decoding with method OAEP """
        
        k = Len(self.kd[0])
        
        c = OS2IP(cipher_text)
        m = DecryptCRT(c, self.kdCRT)
        EM = I2OSP(m, len(cipher_text))
        
        return DecodeOAEP(k, EM, L, Hash, MaskHash)

In [37]:
message = b'The sea and the sky are two symbols of infinity'

keys = RSA_key_generation_CRT(1024)
ke, kdCRT, kd = keys['k_e'], keys['k_d_CRT'], keys['k_d']

In [38]:
public_key = PublicKey(ke)
private_key = PrivateKey(kdCRT, kd)

a = public_key.encode_PKCS(message)
private_key.decode_PKCS(a)

'The sea and the sky are two symbols of infinity'

In [39]:
b = public_key.encode_OAEP(message)
private_key.decode_OAEP(b)

b'The sea and the sky are two symbols of infinity'

Шифрование чисел, кодировка текста.